## Persistent Shock Model and Income Expectations


This module creates an example application extending $\texttt{PersistentShockConsumerType}$ from $\texttt{ConsGenIndShockModel}$. (*Hyperlink to GenIncProcessModel.ipynb)* 

Most simply, it solves a consumption-saving model with neither necessarily fully transitory nor fully permanent, for which persistent income is tracked as a state variable and follows an AR(1) process.

###  But what if beliefs about persistent income differ from actual ?

What if the consumer is having some beliefs about the persistence of his/her income process which differ from the *actual* persistent income?

We can use the class $\texttt{PersistentShockConsumerType}$ as an appropriate model to solve the problem of an identical consumer but for experiencing some beliefs about his income process.

Suppose the consumer *perceived* the persistence of his/her income process differently than what the *actual* one is. We can prepare the parameters to create instances of that class. 

In [ ]:
# Initial imports and notebook setup, click arrow to show
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
from copy import copy


from HARK.ConsumptionSaving.ConsGenIncProcessModel import *
import HARK.ConsumptionSaving.ConsumerParameters as Params

from HARK.utilities import approxUniform, getLorenzShares
from time import clock
mystr = lambda number : "{:.4f}".format(number)

In [ ]:
class PersistentShockConsumerTypeX(PersistentShockConsumerType):
    def getControls(self):
        cLvlNow = np.zeros(self.AgentCount) + np.nan
        MPCnow = np.zeros(self.AgentCount) + np.nan
        for t in range(self.T_cycle):
            these = t == self.t_cycle
            cLvlNow[these] = self.solution[t].cFunc(self.mLvlNow[these],self.pLvlNow[these])
            MPCnow[these]  =self.solution[t].cFunc.derivativeX(self.mLvlNow[these],self.pLvlNow[these])
        self.cLvlNow = cLvlNow
        self.MPCnow  = MPCnow

In [ ]:
# This cell defines a dictionary to make an instance of "persistent idiosyncratic shocks" consumer
BaselineDict = {
        "CRRA": 2.0,                           # Coefficient of relative risk aversion
        "Rfree": 1.03,                         # Interest factor on assets
        "DiscFac": 0.96,                       # Intertemporal discount factor
        "LivPrb" : [0.98],                     # Survival probability
        "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
        "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
        "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
        "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
        "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
        "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
        "T_age" : None,                        # Age after which simulated agents are automatically killed
        "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
        "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
        "aXtraMax" : 30,                       # Maximum end-of-period "assets above minimum" value               
        "aXtraExtra" : [0.005,0.01],           # Some other value of "assets above minimum" to add to the grid
        "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
        "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
        "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
        "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
        "PermShkStd" : [0.1],                  # Standard deviation of log permanent income shocks
        "TranShkStd" : [0.1],                  # Standard deviation of log transitory income shocks
        "UnempPrb" : 0.05,                     # Probability of unemployment while working
        "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
        "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
        "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
        "tax_rate" : 0.0,                      # Flat income tax rate
        "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
        "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
        "CubicBool" : False,                   # Use cubic spline interpolation when True, linear interpolation when False
        "vFuncBool" : True,                    # Whether to calculate the value function during solution    
        "cycles": 0,                           # Make this type have an infinite horizon
        "pLvlPctiles" : np.concatenate(([0.001, 0.005, 0.01, 0.03], np.linspace(0.05, 0.95, num=19),[0.97, 0.99, 0.995, 0.999])),
        "PermGroFac" : [1.0],                  # Permanent income growth factor - long run permanent income growth doesn't work yet  
        "PrstIncCorr": 0.99,                   # Serial correlation coefficient for persistence of income
    }
    
    

In [ ]:
def runRoszypalSchlaffmanExperiment(CorrAct, CorrPcvd, DiscFac_center, DiscFac_spread):
    '''
    Solve and simulate a consumer type who misperceives the extent of serial correlation
    of persistent shocks to income.
    
    Parameters
    ----------
    CorrAct : float
        Serial correlation coefficient for *actual* persistent income
    CorrPcvd : float
        List or array of *perceived* persistent income
    DiscFac_center : float
        A measure of centrality for the distribution of the beta parameter, DiscFac
    DiscFac_spread : float
        A measure of spread or diffusion for the distribution of the beta parameter.


    Returns
    -------
    - AggWealthRatio
    - Lorenz curve for assets
    - Gini coefficient for assets
    - average MPC 
    '''     
    
    # Make a dictionary to construct our consumer type
    ThisDict = copy(BaselineDict)
    ThisDict['PrstIncCorr'] = CorrAct
    
    # Make a 7 point approximation to a uniform distribution of DiscFac, U[center-spread,center+spread]
    param_dist = approxUniform(N=7,bot=DiscFac_center-DiscFac_spread,top=DiscFac_center+DiscFac_spread)
    
    type_list = []

    for i in range(len(param_dist[1])):    
        ThisDict['DiscFac'] = param_dist[1][i]    
        ThisType = PersistentShockConsumerTypeX(**ThisDict)
              
        # Make the consumer type *believe* he will face a different level of persistence
        ThisType.PrstIncCorr = CorrPcvd
        ThisType.updatepLvlNextFunc() # Now he *thinks* E[p_{t+1}] as a function of p_t is different than it is
    
        # Solve the consumer's problem with *perceived* persistence 
        ThisType.solve()
    
        # Make the consumer type experience the true level of persistence during simulation
        ThisType.PrstIncCorr = CorrAct
        ThisType.updatepLvlNextFunc()
    
        # Simulate the agents for many periods, tracking their consumption, MPC, assets, and persistent income
        ThisType.T_sim = 1000
        #ThisType.track_vars = ['cLvlNow','aLvlNow','pLvlNow','MPCnow']
        ThisType.makeShockHistory()
        ThisType.initializeSim()
        ThisType.simulate()
        type_list.append(ThisType)
    
    # Get the most recent simulated values of X = cLvlNow, MPCnow, aLvlNow, pLvlNow for all types   
    cLvl_all = np.concatenate([this_type.cLvlNow[-1,:] for this_type in type_list])
    aLvl_all = np.concatenate([this_type.aLvlNow[-1,:] for this_type in type_list])
    MPC_all = np.concatenate([this_type.MPCnow[-1,:] for this_type in type_list])
    pLvl_all = np.concatenate([this_type.pLvlNow[-1,:] for this_type in type_list])
    
    
    AggWealthRatio = np.mean(aLvl_all) / np.mean(pLvl_all)
    
    Lorenz = getLorenzShares(np.linspace(0.001,0.999,201))
    plt.plot(Lorenz[0],Lorenz[1])
    
    #Gini = np.mean(Lorenz[0][1:-1] - Lorenz[1][1:-1])
    Gini = np.mean(Lorenz[0][:] - Lorenz[1][:])
    
    Avg_MPC = calcSubpopAvg(MPC_all, pLvl_all, cutoffs=[(0.0,0.2), (0.2,0.4),  (0.4,0.6), (0.6,0.8), (0.8,1.0)])
    
    return AggWealthRatio, Lorenz, Gini, Avg_MPC


The user needs to call the function $\texttt{runRoszypalSchlaffmanExperiment}$ with specific values for $\texttt{CorrAct}$ and $\texttt{CorrPcvd}$ to solve the model accordingly.



In [ ]:
# Call the function with test values and parameter estimates for (CorrAct, CorrPcvd, DiscFac_center, DiscFac_spread)
AggWealthRatio, Lorenz, Gini, Avg_MPC = runRoszypalSchlaffmanExperiment(0.97, 0.9831, 0.986609223266, 0.00853886395698)

print('The aggregate wealth to income ratio is ' + str(AggWealthRatio))
print('The Lorenz curve for assests is ' + str(Lorenz))
print('The Gini Coefficient for assests is ' + str(Gini))
print('The average MPC by income quantile is ' + str(Avg_MPC))